# Inference and Visualization Tutorial

Welcome to the inference and visualization notebook! At this point, you should have a trained model and tiles to run inference. In this notebook we will run inference on a slide and visualize the results. Here are the steps we will review:

- Run inference with a trained model.
- Visualize the inference results

## Run inference with a trained model

Often tissue-based analysis on whole slide images benefit from annotations provided by expert pathologists. However, having pathologists annotate 1000s of slides is very time consuming and expensive. To overcome this bottleneck, it is common to have pathologist annotate a subset of the slides, and use that dataset to train a model. This model is then used to label the rest of the dataset.

In the model training notebook, we trained a ResNet-18 model on a subset of our slides with the annotated regions and labels. We will now use this trained model and the prepared tiles from the test slide to run the inference step.

In [ ]:
import os
HOME = os.environ['HOME']

In [ ]:
env DATASET_URL=file:///$HOME/vmount/PRO-12-123/

In [ ]:
!infer_tiles --help

`infer_tiles` CLI takes in details on your trained model, and loads the tiles data for inference.

First we need to copy the model checkpoint into the `classifier` directory where we have a `hubconf.py` file that defines the model settings. 



In [ ]:
! cp ../PRO-12-123/tissue_classifier_results/checkpoint_1.pt ~/vmount/classifier/model.checkpoint 

Next, we need to generate tiles for the slides we want to compute predictions on. Previously, we only generated tiles for portions of the whole slide that contained annotations. Now, we want to tile the entire slide and only remove tiles that have background/glass and keep all of tiles containing tissue. 

Just like before, the pipeline is to first `generate_tiles` for a particular slide, then to run `detect_tissue` to remove glass using an otsu threshold, and then to `save_tiles`, which we can use for inference. 

In [ ]:
!generate_tiles \
file:~/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
--tile_size 128 --requested_magnification 20 \
-o ~/vmount/PRO-12-123/tiling/inference/tiles

In [ ]:
!detect_tissue \
~/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
~/vmount/PRO-12-123/tiling/inference/tiles \
--requested_magnification 2 \
--filter_query "otsu_score > 0.1" \
-o ~/vmount/PRO-12-123/tiling/inference/detect

In [ ]:
!save_tiles \
~/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
~/vmount/PRO-12-123/tiling/inference/detect \
--num_cores 16 --batch_size 200 --dataset_id PRO_TILES_INFERENCE \
-o ~/vmount/PRO-12-123/tiling/inference/saved_tiles

Now that we have a set of saved tiles, we can run inference by specifying the dataset of saved tiles and some parameters about our model. 

In [ ]:
!infer_tiles ~/vmount/PRO-12-123/datasets/PRO_TILES_INFERENCE/ \
--output_dir ../PRO-12-123/sample_tiles_inference \
--hub_repo_or_dir ~/vmount/classifier \
--model_name 'tissue_classifier' \
--num_cores 4 \
--batch_size 16 \
-kw "{'network':'torchvision.models.resnet18', 'num_labels': 3}"

The output of the inference is saved in a paquet file. Let's take a look at the results.


In [ ]:
!ls -lhtr ../PRO_12-123/sample_tiles_inference 

In [ ]:
import pandas as pd

results = pd.read_parquet("../PRO_12-123/sample_tiles_inference/tile_scores_and_labels_pytorch_inference.parquet")

display(results)

## Visualize the inference results 

Now we will visualize the inference results. **visualize_tiles_png** CLI creates heatmaps based on the scores, and saves the thumbnail images in png format.


In [ ]:
!visualize_tiles_png --help

If you want to evaluate your model results in detail, it is desirable to review the results and images in high-magnification.
We use [Digital Slide Archive (DSA)](https://digitalslidearchive.github.io/digital_slide_archive/) viewer to examine the high resolution image and results. DSA is a web-based platform and this enables us to easily share the images and model results with other researchers via a link.

A set of CLIs are available to help you convert your pathologist or model-generated annotations and push them to DSA. Please refer to the `dsa-tools.ipynb` notebook for more details.


In [ ]:
!visualize_tiles_png \
~/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
../PRO-12-123/sample_tiles_inference/tile_scores_and_labels_pytorch_inference.parquet \
--output_dir ../PRO-12-123/sample_tiles_viz \
--plot_labels 0 \
--requested_magnification 1


In [ ]:
from IPython.display import Image

Image('../PRO-12-123/sample_tiles_viz/tile_scores_and_labels_visualization_0.png')

Congratulations on completing the inference and visualization notebook! To view the end-to-end pipeline of the tiling workflow, please checkout the end-to-end notebook.